needed to reload modules, otherwise you will have nan errors

In [1]:
#%load_ext autoreload
#%autoreload 2

### Data

In [2]:
from data_process_and_get_mask import create_mask
from get_data import pbmc_train, pbmc_val

tf_list = ['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7',
       'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A',
       'STAT5B','STAT6','GTF2I','RUNX1', 'MYOD1', 'SOX2','KLF4', 'THAP11','ADNP2', 'AEBP1','AHRR', 'ALX4','APBB1','APEX1', 'ARHGAP35','ARID1A']

#['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A','STAT5B', 'STAT6']
#["GTF2I","GTF3A","NRF1","ELF1","STAT1","STAT2","IRF9","STAT3","STAT4","STAT5A","STAT5B","IRF3","IRF7","IRF1","IRF5","IRF8"]

mask, train, valid = create_mask(pbmc_train, pbmc_val, tf_list)
print("Printing mask:")
print(mask)
print("\nPrinting train:")
print(train)
print("\nPrinting valid:")
print(valid)

Printing mask:
shape: (471, 32)
┌────────┬────────┬────────┬───────┬───┬────────┬───────┬───────┬───────────────┐
│ target ┆ STAT5A ┆ STAT5B ┆ STAT4 ┆ … ┆ ARID1A ┆ ADNP2 ┆ APBB1 ┆ unannotated_1 │
│ ---    ┆ ---    ┆ ---    ┆ ---   ┆   ┆ ---    ┆ ---   ┆ ---   ┆ ---           │
│ str    ┆ f64    ┆ f64    ┆ f64   ┆   ┆ f64    ┆ f64   ┆ f64   ┆ i32           │
╞════════╪════════╪════════╪═══════╪═══╪════════╪═══════╪═══════╪═══════════════╡
│ IL2    ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ CCND1  ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ PRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IFNG   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ …      ┆ …      ┆ …      ┆ …     ┆ … ┆ …      ┆ …     ┆ …     ┆ …             │
│ ZNF181 ┆ 0.0    ┆ 0.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 

In [3]:
# model training
#vega, vega_losses, vega_klds, vega_mses = trainVEGA(vega, train, epochs = 20, beta = 0.0001) #takes about 2 mins on GPU # epoch 100

now use the common training functions

In [4]:
mask

target,STAT5A,STAT5B,STAT4,IRF1,STAT3,RUNX1,GTF2I,IRF2,STAT1,IRF8,STAT6,APEX1,IRF4,MYOD1,ARHGAP35,IRF3,IRF5,ALX4,STAT2,IRF7,KLF4,IRF9,AHRR,AEBP1,SOX2,THAP11,IRF6,ARID1A,ADNP2,APBB1,unannotated_1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""IL2""",1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IRF1""",1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""CCND1""",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
"""PRF1""",1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IFNG""",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZNF181""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF230""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF302""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [5]:
from training.training import run_vega_model
run_vega_model(
    model_type="vega",
    train_data=train,
    valid_data=valid,
    mask_df=mask,
    epochs=30,
    N=10
)

epoch:  0  train_loss:  83.97045   valid_loss:  57.35237
epoch:  10  train_loss:  41.03598   valid_loss:  33.39432
epoch:  20  train_loss:  37.82006   valid_loss:  30.682905
epoch:  0  train_loss:  85.77435   valid_loss:  63.373245
epoch:  10  train_loss:  41.576485   valid_loss:  34.028824
epoch:  20  train_loss:  38.172825   valid_loss:  31.280102
epoch:  0  train_loss:  81.15408   valid_loss:  50.72145
epoch:  10  train_loss:  39.86297   valid_loss:  32.52399
epoch:  20  train_loss:  37.6357   valid_loss:  30.73659
epoch:  0  train_loss:  82.99302   valid_loss:  53.191326
epoch:  10  train_loss:  40.722507   valid_loss:  33.205173
epoch:  20  train_loss:  38.184315   valid_loss:  32.13211
epoch:  0  train_loss:  83.08233   valid_loss:  51.3127
epoch:  10  train_loss:  40.713707   valid_loss:  32.855022
epoch:  20  train_loss:  37.644836   valid_loss:  30.665873
epoch:  0  train_loss:  84.491806   valid_loss:  56.651585
epoch:  10  train_loss:  40.406445   valid_loss:  32.29798
epoch

(VEGA(
   (decoder): DecoderVEGA(
     (sparse_layer): Sequential(
       (0): SparseLayer()
     )
   )
   (encoder): Encoder(
     (encoder): Sequential(
       (0): Linear(in_features=471, out_features=800, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.3, inplace=False)
       (3): Linear(in_features=800, out_features=800, bias=True)
       (4): ReLU()
       (5): Dropout(p=0.3, inplace=False)
     )
     (mu): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
     (sigma): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
   )
 ),
           STAT5A    STAT5B     STAT4      IRF1     STAT3     RUNX1     GTF2I  \
 IL2    -0.000042  0.000096  0.000000  0.000000 -0.000096  0.000003  0.000000   
 IRF1   -0.000547  0.007368  0.002736 -0.001101 -0.000275  0.000000  0.000000   
 CCND1  -0.000112 -0.000099  0.000000  0.000126 -0.000131 

bayes

In [6]:
from training.training import run_vega_model
run_vega_model(
    model_type="bayes",
    train_data=train,
    valid_data=valid,
    mask_df=mask,
    epochs=50,
    N=1
)

AttributeError: 'Tensor' object has no attribute 'drop'